In [1]:
# def calc_hard_dice(y_true, y_pred):
#     numerator = 2.0 * np.sum(np.round(y_true)*np.round(y_pred))
#     denominator = np.sum(np.round(y_true)) + np.sum(np.round(y_pred))
#     if denominator == 0:
#        return 1.0
#     else:
#        return numerator / denominator

# def calc_dice(y_true, y_pred, smooth=1):
#     """
#     Sorensen Dice coefficient
#     """
#     numerator = 2.0 * np.sum(y_true * y_pred) + smooth
#     denominator = np.sum(y_true) + np.sum(y_pred) + smooth
#     coef = numerator / denominator
    
#     print (numerator, denominator, coef)

#     return coef

def calc_hard_dice(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    
    numerator = 2.0 * np.sum(np.round(y_true)*np.round(y_pred))
    denominator = np.sum(np.round(y_true)) + np.sum(np.round(y_pred))
    if denominator == 0:
       return 1.0
    else:
       return numerator / denominator

def calc_dice(y_true, y_pred, smooth=1):
    """
    Sorensen Dice coefficient
    """
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    
    numerator = 2.0 * np.sum(y_true * y_pred) + smooth
    denominator = np.sum(y_true) + np.sum(y_pred) + smooth
    coef = numerator / denominator

    return coef

def dice_coef(y_true, y_pred, axis=(1, 2), smooth=1):
    """
    Sorenson (Soft) Dice
    \frac{  2 \times \left | T \right | \cap \left | P \right |}{ \left | T \right | +  \left | P \right |  }
    where T is ground truth mask and P is the prediction mask
    """
    intersection = tf.reduce_sum(y_true * y_pred, axis=axis)
    union = tf.reduce_sum(y_true + y_pred, axis=axis)
    numerator = tf.constant(2.) * intersection + smooth
    denominator = union + smooth
    coef = numerator / denominator

    return tf.reduce_mean(coef)

def soft_dice_coef(target, prediction, axis=(1, 2), smooth=0.01):
    """
    Sorenson (Soft) Dice  - Don't round the predictions
    \frac{  2 \times \left | T \right | \cap \left | P \right |}{ \left | T \right | +  \left | P \right |  }
    where T is ground truth mask and P is the prediction mask
    """

    intersection = tf.reduce_sum(target * prediction, axis=axis)
    union = tf.reduce_sum(target + prediction, axis=axis)
    numerator = tf.constant(2.) * intersection + smooth
    denominator = union + smooth
    coef = numerator / denominator

    return tf.reduce_mean(coef)

def dice_coef_loss(target, prediction, axis=(1, 2), smooth=1.):
    """
    Sorenson (Soft) Dice loss
    Using -log(Dice) as the loss since it is better behaved.
    Also, the log allows avoidance of the division which
    can help prevent underflow when the numbers are very small.
    """
    intersection = tf.reduce_sum(prediction * target, axis=axis)
    p = tf.reduce_sum(prediction, axis=axis)
    t = tf.reduce_sum(target, axis=axis)
    numerator = tf.reduce_mean(intersection + smooth)
    denominator = tf.reduce_mean(t + p + smooth)
    dice_loss = -tf.log(2.*numerator) + tf.log(denominator)

    return dice_loss


def combined_dice_ce_loss(y_true, y_pred, axis=(1, 2), smooth=1.,
                          weight=0.9):
    """
    Combined Dice and Binary Cross Entropy Loss
    """
    return weight*dice_coef_loss(y_true, y_pred, axis, smooth) + \
        (1-weight)*K.losses.binary_crossentropy(y_true, y_pred)


def plot_results(model, imgs_validation, msks_validation, img_no, png_directory):
    """
    Calculate the Dice and plot the predicted masks for image # img_no
    """

    img = imgs_validation[[img_no], ]
    msk = msks_validation[[img_no], ]

    pred_mask = model.predict(img)

    dice_score = calc_dice(pred_mask, msk)

    print("{:.4f}, {:.4f}".format(dice_score, calc_hard_dice(pred_mask, msk)))


In [2]:
# parser = argparse.ArgumentParser(
#     description="Inference example for trained 2D U-Net model on BraTS.",
#     add_help=True, formatter_class=argparse.ArgumentDefaultsHelpFormatter)

# parser.add_argument("--data_path", default=settings.DATA_PATH,
#                     help="the path to the data")
# parser.add_argument("--data_filename", default=settings.DATA_FILENAME,
#                     help="the HDF5 data filename")
# parser.add_argument("--output_path", default=settings.OUT_PATH,
#                     help="the folder to save the model and checkpoints")
# parser.add_argument("--inference_filename", default=settings.INFERENCE_FILENAME,
#                     help="the Keras inference model filename")

# parser.add_argument("--intraop_threads", default=settings.NUM_INTRA_THREADS,
#                     type=int, help="Number of intra-op-parallelism threads")
# parser.add_argument("--interop_threads", default=settings.NUM_INTER_THREADS,
#                     type=int, help="Number of inter-op-parallelism threads")
import os

import numpy as np
import tensorflow as tf
import keras as K
import settings
import argparse
import h5py

import matplotlib.pyplot as plt

args = argparse.Namespace()
args.data_path="/home/bduser/tony/data/decathlon/"
args.data_filename="240x240/decathlon_brats.h5"
args.output_path="/home/bduser/tony/unet_tiling/"
#args.inference_filename="unet_decathlon_4_8814_128x128_randomcrop.hdf5"
args.inference_filename="unet_decathlon_4_8814_128x128_randomcrop-any-input.h5"
args.inference_filename="unet_decathlon_240x240_dice0820.hdf5"
args.intraop_threads=5
args.interop_threads=2

CONFIG = tf.ConfigProto(
    inter_op_parallelism_threads=args.interop_threads,
    intra_op_parallelism_threads=args.intraop_threads)

SESS = tf.Session(config=CONFIG)
K.backend.set_session(SESS)

data_fn = os.path.join(args.data_path, args.data_filename)
model_fn = os.path.join(args.output_path, args.inference_filename)

# Load data
print("Loading Data... ")
df = h5py.File(data_fn, "r")
imgs_validation = df["imgs_validation"]
msks_validation = df["msks_validation"]
print("Data loaded successfully from: " + data_fn)

# Load model
print("Loading Model... ")
model = K.models.load_model(model_fn, custom_objects={
    "combined_dice_ce_loss": combined_dice_ce_loss,
    "dice_coef_loss": dice_coef_loss,
    "soft_dice_coef": soft_dice_coef,
    "dice_coef": dice_coef})
print("Model loaded successfully from: " + model_fn)

# Create output directory for images
png_directory = "inference_examples"
if not os.path.exists(png_directory):
    os.makedirs(png_directory)

# Plot some results
# The plots will be saved to the png_directory
# Just picking some random samples.
indicies_validation = [40, 61, 400, 1100, 4385,
                       5566, 5673, 6433, 7864, 8899, 9003, 9722, 10591]




Using TensorFlow backend.


Loading Data... 
Data loaded successfully from: /home/bduser/tony/data/decathlon/240x240/decathlon_brats.h5
Loading Model... 
Model loaded successfully from: /home/bduser/tony/unet_tiling/unet_decathlon_240x240_dice0820.hdf5


In [3]:

print("Number of validation images: ", imgs_validation.shape[0])

Number of validation images:  12090


In [4]:
img_no = 11
full_img = imgs_validation[[img_no], ]
full_msk = msks_validation[[img_no], ]
print("Full image shape: ", full_img.shape)
print("Full mask shape: ", full_msk.shape)

Full image shape:  (1, 240, 240, 4)
Full mask shape:  (1, 240, 240, 1)


In [5]:
pred_mask = model.predict(full_img)
print("Pred mask shape: ", pred_mask.shape)

dice_score = calc_dice(pred_mask, full_msk)
hard_dice_score = calc_hard_dice(pred_mask, full_msk)
print("Dice Score:", dice_score)
print("Hard Dice Score:", hard_dice_score)

Pred mask shape:  (1, 240, 240, 1)
Dice Score: 0.9999999999970315
Hard Dice Score: 1.0


In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
MRImages (InputLayer)           (None, None, None, 4 0                                            
__________________________________________________________________________________________________
encodeAa (Conv2D)               (None, None, None, 3 1184        MRImages[0][0]                   
__________________________________________________________________________________________________
encodeAb (Conv2D)               (None, None, None, 3 9248        encodeAa[0][0]                   
__________________________________________________________________________________________________
poolA (MaxPooling2D)            (None, None, None, 3 0           encodeAb[0][0]                   
__________________________________________________________________________________________________
encodeBa (

In [7]:
def calc_dice_scores(pred_mask, msk):
    dice_score = calc_dice(pred_mask, msk)
    hard_dice_score = calc_hard_dice(pred_mask, msk)
    return dice_score, hard_dice_score 

def print_avg_dice_scores(dice_results):
    dice_results = np.asarray(dice_results)
    avg_dice = np.average(dice_results[:,1])
    avg_hard_dice = np.average(dice_results[:,2])
    print("Avg Dice Score: {:.4f},\nAvg Hard Dice Score: {:.4f}".format(avg_dice, avg_hard_dice))    
    

In [8]:
from tqdm import tqdm

dice_results = []
#for img_id in tqdm(range(240,280)):
for img_id in tqdm(range(imgs_validation.shape[0])):

    full_img = imgs_validation[[img_id], ]
    full_msk = msks_validation[[img_id], ]
    
    pred_mask = model.predict(full_img)
    dice_score, hard_dice_score = calc_dice_scores(pred_mask, full_msk)
    dice_results.append([img_id, dice_score, hard_dice_score])

print_avg_dice_scores (dice_results) 


100%|██████████| 12090/12090 [15:55<00:00, 13.92it/s]

Avg Dice Score: 0.8396,
Avg Hard Dice Score: 0.8360
